In [ ]:
site_gen_urls = {
    "gen_1": "https://projectpokemon.org/home/docs/spriteindex_148/3d-models-generation-1-pok%C3%A9mon-r90/",
    "gen_2": "https://projectpokemon.org/home/docs/spriteindex_148/3d-models-generation-2-pok%C3%A9mon-r91/",
    "gen_3": "https://projectpokemon.org/home/docs/spriteindex_148/3d-models-generation-3-pok%C3%A9mon-r92/",
    "gen_4": "https://projectpokemon.org/home/docs/spriteindex_148/3d-models-generation-4-pok%C3%A9mon-r93/",
    "gen_5": "https://projectpokemon.org/home/docs/spriteindex_148/3d-models-generation-5-pok%C3%A9mon-r94/",
    "gen_6": "https://projectpokemon.org/home/docs/spriteindex_148/3d-models-generation-6-pok%C3%A9mon-r95/",
    "gen_7": "https://projectpokemon.org/home/docs/spriteindex_148/3d-models-generation-7-pok%C3%A9mon-r96/",
}

save_url = r"C:\repos\poke_gen\data\scraped_images"

In [ ]:
import requests
from bs4 import BeautifulSoup


def scrape_images_from_table(website_url):
    image_srcs = {"front": [], "back": []}
    table_type = ["front", "back"]
    try:
        # Send a GET request to the website
        response = requests.get(website_url)

        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            # Parse the HTML content of the page
            soup = BeautifulSoup(response.content, "html.parser")

            # Find the table element(s)
            tables = soup.find_all("table")

            # Iterate through each table
            for table_type, table in zip(table_type, tables):
                # Find the tbody element(s)
                tbodies = table.find_all("tbody")

                # Iterate through each tbody
                for tbody in tbodies:
                    # Find all tr elements within the tbody
                    rows = tbody.find_all("tr")

                    # Iterate through each row
                    for row in rows:
                        # Find all td elements within the row
                        cells = row.find_all("td")

                        # Iterate through each cell
                        for cell in cells:
                            # Find all 'a' elements within the cell
                            images = cell.find_all("img")

                            # Iterate through each 'img' element and get the 'src' attribute
                            for img in images:
                                img_src = img.get("src")
                                if img_src:
                                    image_srcs[table_type].append(img_src)

    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")

    return image_srcs

In [ ]:
url_dict = {}
for gen, url in site_gen_urls.items():
    urls = scrape_images_from_table(url)
    url_dict[gen] = urls

In [ ]:
url_dict

In [ ]:
images_url_list = []
for gen_name, gen_dict in url_dict.items():
    for type_name, link_list in gen_dict.items():
        for item in link_list:
            images_url_list.append(item)

In [ ]:
from urllib.parse import urlparse

url = "https://projectpokemon.org/images/normal-sprite/bulbasaur.gif"

# Parse the URL to extract the path
parsed_url = urlparse(url)

# Extract the path
path = parsed_url.path

print(path)

In [ ]:
image_names = []
for gen_name, gen_dict in url_dict.items():
    for type_name, link_list in gen_dict.items():
        for link in link_list:
            # Parse the URL to extract the path and remove slashes
            path = urlparse(link).path.replace("/", "-")

            # Construct the image_name
            image_name = f"{gen_name}-{type_name}{path}"

            # Append the image_name to the list
            image_names.append(image_name)

In [ ]:
# Define the CSV file name
import csv
import random
import time

csv_filename = f"{save_url}\\image_data.csv"
image_names = []
# Create and write to the CSV file
with open(csv_filename, mode="w", newline="") as csv_file:
    writer = csv.writer(csv_file)

    # Write the header row
    writer.writerow(["img_path", "img_gen", "type"])

    # Write the data rows
    for gen_name, gen_dict in url_dict.items():
        for type_name, link_list in gen_dict.items():
            for link in link_list:
                try:
                    time.sleep(random.uniform(0.05, 0.15))
                    response = requests.get(link)
                    if response.status_code == 200:
                        image_data = response.content
                        local_image_path = f"{save_url}\\{image_name}"
                        with open(local_image_path, "wb") as local_file:
                            local_file.write(image_data)
                    else:
                        print(f"Failed to retrieve the image. Status code: {response.status_code}")
                except requests.exceptions.RequestException as e:
                    print(f"Error: {e}")
                    pass

                # Parse the URL to extract the path and remove slashes
                path = urlparse(link).path.replace("/", "-")
                # Construct the image_name
                image_name = f"{gen_name}-{type_name}{path}"

                # Append the image_name to the list
                writer.writerow([f"{save_url}\\{image_name}", gen_name, type_name])
                image_names.append(image_name)